In [ ]:
import cv2
import sys
import os
import numpy as np
import logging
import glob
import insightface
from insightface.app import FaceAnalysis
from cpu_searcher import numpy_searcher

In [ ]:
app = FaceAnalysis()
app.prepare(ctx_id=0, det_size=(640, 640))

In [ ]:
img_pathes = glob.glob("/home/avs/Codes/face_db/CBA/*/*.jpg")
img_pathes.extend(glob.glob("/home/avs/Codes/face_db/CBA/*/*.jpeg"))

In [ ]:
def path_2_img_key(path):
    img_name = os.path.basename(path)
    politican_name = os.path.basename(os.path.dirname(path))
    return '{}+{}'.format(politican_name, img_name)

def read_image_to_data(image_path):
    image_data = cv2.imread(image_path)
    if image_data is None:
        logging.error("== read invalid data ", image_path)
        return
    
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    return image_data

In [ ]:
img_keys = map(path_2_img_key, img_pathes)
img_datas = list(map(read_image_to_data, img_pathes))

In [ ]:
searcher = numpy_searcher(1000, 512)
for img_data,img_key in zip(img_datas, img_keys):
    faces = app.get(img_data)
    if len(faces) != 1:
        print("==invalid len {} for image {}".format(len(faces), img_key))
        continue
    else:
        feat = faces[0].normed_embedding.tolist()
        searcher.update(img_key, feat)
    

In [ ]:
test_images = glob.glob("/home/avs/Codes/face_db/CBA/*/*.png")

In [ ]:
test_image_datas = list(map(read_image_to_data, test_images))

In [ ]:
for test_image_data, test_path in zip(test_image_datas, test_images):
    faces = app.get(test_image_data)
    test_feats = []
    for face in faces:
        test_feat = face.normed_embedding.tolist()
        test_feats.append(test_feat)
    test_feats = np.array(test_feats)
    print('== test path {} with {} face'.format(test_path, len(faces)))
    topk_keys = searcher.topk(test_feats, topk=1)
    print("== search result ", topk_keys)

In [ ]:
topk_keys